# MODELADO

## Dense Newtork


<hr>

<code> **Proyecto de Datos II** </code>

## Índice

- [Importación de los datos](#importación-de-los-datos)
- [Preprocesamiento](#preprocesamiento)
- [Entrenamiento](#entrenamiento)
- [Análisis del modelo](#análisis-del-modelo)
- [Registro del modelo en MLflow](#registro-del-modelo-en-mlflow)


In [3]:
import time
import mlflow
import pandas as pd
from evaluation.evaluator import Evaluator

SEED = 22 # replicabilidad

# =====================================
MODEL_NAME = "dense_network"
# =====================================

## Importación de los datos

Se ha tomado del conjunto de datos de entrenamiento como validación del 1 al 14 de enero (los mensajes más recientes)

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import keras_tuner as kt

# Cargamos los datos
df_train = pd.read_parquet("../data/train.parquet")
df_test = pd.read_parquet("../data/test.parquet")

# Separamos en variable objetivo y features
X_train_full = df_train.drop(columns="takeoff_time")
y_train_full = df_train["takeoff_time"]

X_test = df_test.drop(columns="takeoff_time")
y_test = df_test["takeoff_time"]

# Convertimos columna timestamp a datetime
X_train_full["timestamp"] = pd.to_datetime(X_train_full["timestamp"])

# Dividimos entre train y validation basado en fecha para evitar contaminación de futuro
split_date = pd.to_datetime("2025-01-01")
train_mask = X_train_full["timestamp"] < split_date
val_mask = X_train_full["timestamp"] >= split_date

X_train = X_train_full[train_mask].drop(columns=["timestamp", "icao", "callsign"])
y_train = y_train_full[train_mask]

X_val = X_train_full[val_mask].drop(columns=["timestamp", "icao", "callsign"])
y_val = y_train_full[val_mask]

X_test = X_test.drop(columns=["timestamp", "icao", "callsign"])

In [8]:
X_train.shape, X_val.shape, X_test.shape

((98645, 57), (25088, 57), (27791, 57))

## Preprocesamiento

Se han normalizado las variables numéricas y aplicado One-Hot Encoding a las variables categóricas.

In [10]:
# - One-hot encoding -
X_train = pd.get_dummies(X_train, drop_first=True)
X_val = pd.get_dummies(X_val, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Alineamos columnas por si falta alguna categoría
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Escalamos las variables numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

Asimismo, también le hemos realizado una transformación logarítmica a la variable respuesta, lo cual mejora ligeramente los resultados del modelo.

In [6]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# 1. Función para transformar el target (log1p)
def preprocess_target(y):
    """Aplica transformación logarítmica a los datos."""
    y_np = y.values.reshape(-1, 1)
    y_log = np.log1p(y_np)
    return y_log

# 2. Función para revertir el preprocesamiento
def inverse_preprocess_target(y_scaled, scaler):
    """Revierte MinMaxScaler y la transformación logarítmica."""
    y_log = scaler.inverse_transform(y_scaled)
    y_original = np.expm1(y_log)
    return y_original

y_scaler = MinMaxScaler()

y_train_log = preprocess_target(y_train)
y_train_scaled = y_scaler.fit_transform(y_train_log)

y_val_log = preprocess_target(y_val)
y_val_scaled = y_scaler.transform(y_val_log)

y_test_log = preprocess_target(y_test)
y_test_scaled = y_scaler.transform(y_test_log)


## Entrenamiento

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import keras_tuner as kt
import time

# Registramos el tiempo de inicio para medir el tiempo total de ejecución
start_time = time.time()

# Definimos la función de construcción del modelo, que será usada por Keras Tuner
def build_model(hp, input_shape):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_shape,)))

    # Seleccionamos aleatoriamente el número de capas ocultas (entre 1 y 3)
    num_layers = hp.Int('num_layers', 1, 3)
    for i in range(num_layers):
        # Añadimos una capa densa con un número de unidades determinado por el optimizador
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=64, max_value=512, step=64),
            activation='relu'
        ))
        
        # Añadimos una capa Dropout para prevenir sobreajuste
        model.add(layers.Dropout(rate=hp.Float(f'dropout_{i}', min_value=0.2, max_value=0.5, step=0.1)))

    # Añadimos la capa de salida con una sola neurona (regresión)
    model.add(layers.Dense(1))

    # Compilamos el modelo usando el optimizador Adam con tasa de aprendizaje y decaimiento ajustables
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', 1e-5, 1e-2, sampling='LOG'),
            decay=hp.Float('lr_decay', 1e-6, 1e-3, sampling='LOG')
        ),
        loss='mse',
        metrics=['mae']
    )
    return model

# Definimos la función que se encargará de ejecutar la búsqueda de hiperparámetros
def tune_model(X_train_scaled, y_train, X_val_scaled, y_val):
    tuner = kt.BayesianOptimization(
        lambda hp: build_model(hp, X_train_scaled.shape[1]),
        objective='val_mae',
        max_trials=30,  # Número máximo de combinaciones que vamos a probar
        directory='kt_tuning_dir_bayesian',
        project_name='takeoff_time',
        overwrite=True
    )

    # Añadimos una parada temprana para evitar sobreentrenamiento
    early_stop = callbacks.EarlyStopping(monitor='val_mae', patience=5, restore_best_weights=True)

    # Ejecutamos la búsqueda de hiperparámetros
    tuner.search(
        X_train_scaled, y_train_scaled,
        validation_data=(X_val_scaled, y_val_scaled),
        epochs=30,
        batch_size=32,
        callbacks=[early_stop],
        verbose=2
    )

    # Obtenemos la mejor combinación de hiperparámetros
    best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_hp, tuner

# Llamamos al proceso de optimización
print("\nBuscando mejores hiperparámetros...")
best_hp, tuner = tune_model(X_train_scaled, y_train, X_val_scaled, y_val)

# Entrenamos el modelo final usando los mejores hiperparámetros encontrados
print("\nEntrenando modelo final con mejores hiperparámetros...")
final_model = build_model(best_hp, X_train_scaled.shape[1])

# Añadimos una parada temprana más estricta para el entrenamiento final
early_stop_final = callbacks.EarlyStopping(monitor='val_mae', patience=10, restore_best_weights=True)

# Entrenamos el modelo final con más épocas
history = final_model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_val_scaled, y_val_scaled),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop_final],
    verbose=2
)

# Registramos el tiempo de fin y calculamos la duración total
end_time = time.time()
execution_time = end_time - start_time

Trial 30 Complete [00h 01m 39s]
val_mae: 0.12479636073112488

Best val_mae So Far: 0.10553476959466934
Total elapsed time: 00h 30m 59s

Entrenando modelo final con mejores hiperparámetros...
Epoch 1/100
3083/3083 - 5s - 1ms/step - loss: 0.0637 - mae: 0.1551 - val_loss: 0.0232 - val_mae: 0.1136
Epoch 2/100
3083/3083 - 4s - 1ms/step - loss: 0.0178 - mae: 0.0999 - val_loss: 0.0228 - val_mae: 0.1119
Epoch 3/100
3083/3083 - 4s - 1ms/step - loss: 0.0126 - mae: 0.0872 - val_loss: 0.0215 - val_mae: 0.1104
Epoch 4/100
3083/3083 - 4s - 1ms/step - loss: 0.0098 - mae: 0.0769 - val_loss: 0.0233 - val_mae: 0.1145
Epoch 5/100
3083/3083 - 4s - 1ms/step - loss: 0.0081 - mae: 0.0697 - val_loss: 0.0240 - val_mae: 0.1158
Epoch 6/100
3083/3083 - 4s - 1ms/step - loss: 0.0069 - mae: 0.0640 - val_loss: 0.0231 - val_mae: 0.1143
Epoch 7/100
3083/3083 - 4s - 1ms/step - loss: 0.0060 - mae: 0.0595 - val_loss: 0.0243 - val_mae: 0.1170
Epoch 8/100
3083/3083 - 4s - 1ms/step - loss: 0.0054 - mae: 0.0561 - val_loss: 0.

## Análisis del modelo

In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Predicciones en el conjunto de entrenamiento
y_train_pred_scaled = final_model.predict(X_train_scaled)
y_train_pred = inverse_preprocess_target(y_train_pred_scaled, y_scaler)
y_train_true = y_train.values.reshape(-1, 1)  # Asegurarse que y_train esté en (n, 1)

# Predicciones en el conjunto de validación
y_val_pred_scaled = final_model.predict(X_val_scaled)
y_val_pred = inverse_preprocess_target(y_val_pred_scaled, y_scaler)
y_val_true = y_val.values.reshape(-1, 1)

# Calculamos las métricas
mae_train = mean_absolute_error(y_train_true, y_train_pred)
rmse_train = np.sqrt(mean_squared_error(y_train_true, y_train_pred))

mae_val = mean_absolute_error(y_val_true, y_val_pred)
rmse_val = np.sqrt(mean_squared_error(y_val_true, y_val_pred))

# Mostramos los resultados
print(f"MAE en entrenamiento: {mae_train:.4f}")
print(f"RMSE en entrenamiento: {rmse_train:.4f}")
print(f"MAE en validación: {mae_val:.4f}")
print(f"RMSE en validación: {rmse_val:.4f}")

3083/3083 ━━━━━━━━━━━━━━━━━━━━ 1s 314us/step
784/784 ━━━━━━━━━━━━━━━━━━━━ 0s 325us/step
MAE en entrenamiento: 51.8880
RMSE en entrenamiento: 76.5503
MAE en validación: 81.3606
RMSE en validación: 134.9141


In [15]:
# ===============================================================
# Generamos predicciones en test (en escala transformada)
y_pred_scaled = final_model.predict(X_test_scaled)

# Invertimos la transformación para obtener las predicciones originales
y_pred = inverse_preprocess_target(y_pred_scaled, y_scaler)

df_test['prediction'] = y_pred.flatten()  # Flatten para que sea una columna 1D
# ===============================================================

869/869 ━━━━━━━━━━━━━━━━━━━━ 0s 322us/step


In [16]:
# Nota: df_test tiene que tener la columna 'prediction'
ev = Evaluator(df_test, MODEL_NAME, mae_val, rmse_val)
report = ev.getReport()
ev.visualEvaluation()

## Registro del modelo en MLflow

In [18]:
mlflow.set_tracking_uri("file:./mlflow_experiments")
mlflow.set_experiment("takeoff_time_prediction")

with mlflow.start_run():

    # - Datos generales -

    # ========================================================================
    mlflow.set_tag("model_type", MODEL_NAME)
    mlflow.set_tag("framework", "tensorflow") # scikit-learn, tensorflow, etc.
    mlflow.set_tag("target_variable", "takeoff_time") # variable respuesta
    mlflow.set_tag("preprocessing", "standard scaler + log y scaler en y") # transformaciones separadas por un +
    mlflow.set_tag("dataset", "original") # indicar si se ha modificado el conjunto de datos
    mlflow.set_tag("seed", SEED) # semilla para replicabilidad
    # ========================================================================
    
    # - Hiperparámetros óptimos -
    
    # =====================================
    if 'best_hp' in locals():
        best_hp_dict = best_hp.values  # Convertir los mejores hiperparámetros a un diccionario

        # Guardar automáticamente los hiperparámetros
        mlflow.log_params(best_hp_dict)
    
    mlflow.log_param("model", MODEL_NAME)
    # =====================================
    
    # - Métricas -

    mlflow.log_metric("execution_time_s", execution_time)

    mlflow.log_metric("mae_val", mae_val)
    mlflow.log_metric("rmse_val", rmse_val)

    mlflow.log_metric("mae_train", mae_train)
    mlflow.log_metric("rmse_train", rmse_train)

    # Registrar métricas globales en test
    for metric_name, value in report["global"].items():
        mlflow.log_metric(f"{metric_name}_test", value)
    
    # Registrar métricas por runway
    for runway, metrics in report["by_runway"].items():
        for metric_name, value in metrics.items():
            mlflow.log_metric(f"{metric_name}_test_runway_{runway}", value)
    
    # Registrar métricas por holding point
    for hp, metrics in report["by_holding_point"].items():
        for metric_name, value in metrics.items():
            mlflow.log_metric(f"{metric_name}_test_hp_{hp}", value)

    # - Modelo -

    # ========================================================================
    # NOTA - Dependiendo de con qué has hecho el modelo esto hay que cambiarlo
    mlflow.keras.log_model(final_model, "model")
    # ========================================================================
    

2025/04/27 15:23:44 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/27 15:23:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [19]:
# - Visualizar experimentos -
# !mlflow ui --backend-store-uri ./mlflow_experiments